<a href="https://colab.research.google.com/github/ramm1024/trabajotitulo/blob/main/POC_TT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [115]:
!pip install -q openai
!pip install -q pattern

In [116]:
from openai import OpenAI
import os
import random
from google.colab import userdata
from google.colab import drive
import pandas as pd
import time
import csv
from io import StringIO
from pattern.text.en import singularize
drive.mount('/content/drive')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KE')
client = OpenAI()
dataset_filename = "dummy12.csv"
df_dataset = pd.read_csv("/content/drive/MyDrive/11vo/poctestdata/"+dataset_filename,sep=",")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [117]:
def set_model(model_choice:str)-> str:
  #https://openai.com/api/pricing/
  if model_choice == "3.5t":
    return "gpt-3.5-turbo"
  elif model_choice == "4":
    return "gpt-4"
  elif model_choice == "4t":
    return  "gpt-4-turbo"
  elif model_choice == "3.5t0125":
    return "gpt-3.5-turbo-instruct"
  elif model_choice == "4o":
    return "gpt-4o"
  elif model_choice == "3.5t0125":
    return "gpt-3.5-turbo-0125"
  else:
    return "gpt-3.5-turbo-0125"#"text-embedding-3-large"
model_choice = set_model("4o")

In [169]:
#input df: df['columna']
#output str: string concatenado por ','
def get_data(df:pd.DataFrame)->str:
  return ','.join(map(str,df.tolist()))

#input data: df con datos a guardar en drive
#output: guarda .csv en drive
def save_output(data:pd.DataFrame):
  with open('/content/drive/MyDrive/11vo/poctestdata/out_'+dataset_filename, 'w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['data_type', 'column_name'])
    for item in data:
      writer.writerow([item,','.join(data[item])])

def sanitize(data_type:str)->str:
  string = singularize(data_type.lower())
  if string == 'email addres':
    return 'email address'
  elif string == 'addres':
    return 'address'
  else:
    return string

def upload_policy(policy_string, filename):
  with open('/content/drive/MyDrive/11vo/poctestdata/' + filename + '.txt', 'w') as f:
    f.write(policy_string)

#input data: str con datos unidos por ;
#output tipo_datos: lista de diccionarios con el tipo de dato encontrado y la cantidad de instancias
#def send_prompt(data:str, model_choice:int)->str:
def send_prompt(mode:str='get_data_types', data:str='', regulation:str='HIPAA', lang:str='english')->str:
  completion = client.chat.completions.create(
    model=model_choice,
    messages=set_prompt(mode=mode,data=data,regulation=regulation,lang=lang)
  )
  return completion.choices[0].message.content

def translate(string:str, lang:str)->str:
  return send_prompt(mode='translate',data=string,lang=lang)

#input data: str con los datos de una columna unidos por ','
#output messages: msg a enviar como prompt para analisis
def set_prompt(mode:str, data:str, regulation:str, lang:str)->str:
  messages = []
  if mode == 'get_data_types':
    messages=[
                      {"role": "system", "content": "You are a data parser"},
                      {"role": "system", "content": "Your datasets come in form of a string separated by semicolons"},
                      {"role": "system", "content": "Yopu will use the dataset to infer the type of personal information that it contains"},
                      {"role": "system", "content": f"Return only the name of the type of personal information"},
                      {"role": "user", "content": f"{data}"},
    ]
  elif mode == 'get_regulation':
    messages=[
                      {"role": "system", "content": "You are a data redactor that creates policies based on a specific regulation"},
                      {"role": "system", "content": f"I need to make a table that describes the rules and guidelines needed to comply with the regulation {regulation}"},
                      {"role": "system", "content": f"Extract all the rules and guidelines found"},
                      {"role": "system", "content": "Adapt this information to a format that follows exactly: Key Points, Description, Compliance Requirements"},
                      {"role": "system", "content": "Include rows specific to the data access rights of said regulation"},
                      {"role": "system", "content": "This table should be as extense as needed and properly separate each function in different rows"},
                      {"role": "system", "content": f"You output will be only text tha represents a csv separated by semicolons"},
                      {"role": "system", "content": f"You are only allowed to print a string with a csv that has the headers: Key Points, Description, Compliance Requirements"},
                      {"role": "user", "content": "Print the output csv string"},
    ]
  elif mode == 'translate':
      messages=[
          {"role": "system", "content": "You are a text translator"},
          {"role": "system", "content": f"Translate the following text to {lang}"},
          {"role": "user", "content": f"{data}"},
    ]
  else:
      raise ValueError(f"Invalid mode: {mode}")
  return messages

In [180]:
class DataType:
  def __init__(self, name:str, description:str, category:str, is_personal_data:bool):
    self.name = name
    self.description = description
    self.category = category
    self.is_personal_data = is_personal_data

  def __str__(self):
    return f"Name: {self.name}\nDescription: {self.description}\nCategory:\
    {self.category}\n\Personal data: {'Yes' if self.is_personal_data else 'No'}"

class DataCategory:
  def __init__(self, name:str):
    self.name = name
    self.data_types:set[DataType] = set()

  def add_data_type(self, data_type:DataType)->None:
    self.data_types.add(data_type)

  def get_data_types(self)->DataType:
    return self.data_types

  def __str__(self,v=False)->None:
    print(f"Name: {self.name}")
    if not v:
      for data_type in self.data_types:
        print(data_type.name)
    else:
      for data_type in self.data_types:
        data_type.print_data_type()

class DataAsset:
  def __init__(self, name:str, source:str, dpo:str):
    self.name = name
    self.dpo = dpo
    self.source = source
    self.data_types:list[DataType] = []
    self.df_data:pd.DataFrame = None
    self.detected_data_types:list[str] = []

  def get_data_categories(self)->set[DataCategory]:
    data_categories = set()
    for data_type in self.data_types:
      data_categories.add(data_type.category)
    return data_categories

  def add_data_type(self, data_type:DataType)->None:
    self.data_types.append(data_type)

  def get_data_types(self)->list[DataType]:
    return self.data_types

  def has_personal_data(self)->bool:
    for data_type in self.data_types:
      if data_type.is_personal_data:
        return True
    return False

  def detect_data_types(self)->None:
    data_types = dict()
    for column_name in self.df_data.columns:
      data_type = send_prompt(data=get_data(self.df_data[column_name]))
      sanitized_data_type = sanitize(data_type)
      if sanitized_data_type not in data_types:
        data_types[sanitized_data_type] = [column_name]
      else:
        data_types[sanitized_data_type].append(column_name)
    self.detected_data_types = data_types

  def get_detected_data_types(self)->list[str]:
    return self.detected_data_types

  def set_df_data(self,df_data:pd.DataFrame)->None:
        self.df_data = df_data

class Process:
  def __init__(self, name:str, data_sources:list[str], purpose:list[str], recipients:list[str],\
               storage_location:list[str], retention_period:list[str]):
    self.name = name
    self.data_sources = data_sources
    self.data_assets:list[DataAsset] = []
    self.purpose = purpose
    self.recipients = recipients
    self.storage_location = storage_location
    self.retention_period = retention_period

  def get_data_assets(self):
    return self.data_assets

  def add_data_asset(self, data_asset:DataAsset):
    self.data_assets.append(data_asset)

  def __str__(self):
        return f"Name: {self.name}\nData Sources: {self.data_sources}\nData Assets: {self.data_assets}\nPurpose: {self.purpose}"


class Regulation:
  def __init__(self):
    self.name:str = ''
    self.df_cumplimiento:pd.DataFrame = None

  def set_cumplimiento(self,name:str):
    csv = send_prompt(mode='get_regulation',regulation=name)
    df_cumplimiento = pd.read_csv(StringIO(csv),sep=";",header=1)
    df_cumplimiento = df_cumplimiento.rename(columns={df_cumplimiento.columns[0]: 'Key Points',\
                                                      df_cumplimiento.columns[1]: 'Description',\
                                                      df_cumplimiento.columns[2]: 'Compliance Requirements'},\
                                              inplace=False)
    self.df_cumplimiento = df_cumplimiento
    self.name = name

class PrivacyPolicy:
    def __init__(self):
        self.dpo = dpo = ''
        #CONSIDERAR BORRAR
        self.data_types:dict[list] = {}
        self.detected_data_types:list[str] = []
        self.gdpr_mapping:dict[Process] = {}
        self.regulations:dict[Regulation] = {}
        #CONSIDERAR BORRAR
        self.df_datasets:list[pd.DataFrame] = None
        self.privacy_policy_string:str = ''

    def set_dpo(self,dpo:str):
        self.dpo = dpo

    def get_dpo(self):
        return self.dpo

    def set_detected_data_types(self):
        data_types = dict()
        for process in self.gdpr_mapping:
          i = 0
          for data_asset in self.gdpr_mapping[process].data_assets:
            self.gdpr_mapping[process].data_assets[i].detect_data_types()
            self.detected_data_types.append(self.gdpr_mapping[process].data_assets[i].get_detected_data_types())
            i += 1

    def get_detected_data_types(self):
        return self.detected_data_types

    def get_all_data_types(self):
      all_data_types = set
      for process in self.gdpr_mapping:
        for data_type in self.gdpr_mapping[process].personal_data_types:
          all_data_types.add(data_type)
      return all_data_types

    def get_gdpr_mapping(self):
        return self.gdpr_mapping

    def set_regulation(self,regulation:str):
      self.regulations.set_cumplimiento(regulation)
      #self.regulations.name = regulations
      #self.regulations.df_cumplimiento = self.set_cumplimiento(regulations)

    def add_process(self, process:Process):
      self.gdpr_mapping[process.name] = process
      #self.gdpr_mapping[process.name] = process.purpose

    def generate_privacy_policy(self, lang:str='english')->str:
        # Check for necessary columns in the DataFrame
        required_columns = ['Description', 'Key Points', 'Compliance Requirements']
        for regulation in self.regulations:
          if not all(column in self.regulations[regulation].df_cumplimiento.columns for column in required_columns):
              raise ValueError(f"DataFrame must contain the following columns: {required_columns}")

        # Start with an introductory section
        policy = "Data Privacy Policy\n"
        policy += "=" * 20 + "\n\n"
        policy += "This policy outlines our commitment to protecting personal data and complying with data privacy regulations.\n\n"

        # Generate sections for each regulations
        for regulation in self.regulations:
          policy += f"Regulation: {regulation}\n"
          policy += "-" * 20 + "\n"
          for index,row in self.regulations[regulation].df_cumplimiento.iterrows():
            if pd.isnull(row['Compliance Requirements']) and pd.isnull(row['Description']):
              policy += f"\nControl a realizar: {row['Key Points']}\n"
            else:
              if row['Key Points'].lower() != 'nan': policy += f"Key Point: {row['Key Points']}\n"
              if row['Compliance Requirements'].lower() != 'nan': policy += f"Compliance Requirements: {row['Compliance Requirements']}\n"
              if row['Description'].lower() != 'nan': policy += f"Description:\n{row['Description']}\n"
              policy += "\n" + "-" * 20 + "\n"

        policy += "Contact Information:\n"
        policy += "If you have any questions about this policy, please contact us at [your contact information].\n\n"

        policy += "Policy Review and Updates:\n"
        policy += "We may update this policy from time to time to reflect changes in our practices or legal requirements. Please check back periodically for updates.\n"

        #omegalul
        policy += self.policy_data_types()
        policy+= self.print_gdpr_mapping()
        if lang != 'english':
          policy = translate(policy,lang)
        self.privacy_policy_string = policy

    def policy_data_types(self)->str:
        # Start generating the policy
        policy = "Data Privacy Policy\n"
        policy += "=" * 20 + "\n\n"
        policy += "This policy outlines our commitment to protecting personal and sensitive data types and complying with data privacy regulations.\n\n"

        # Add sections based on the data types
        policy += "Data Types Covered\n"
        policy += "-" * 20 + "\n"
        for data_type in self.detected_data_types:
            policy += f"- {data_type}\n"
        policy += "\n"
        return policy

    def print_gdpr_mapping(self):
      gdpr_mapping_string = "="*10+" GDPR - Process Data Mapping"+"="*10+'\n'
      for process in self.gdpr_mapping:
        gdpr_mapping_string += f"Process: {process}\n"
        gdpr_mapping_string += f"Data asset: {self.gdpr_mapping[process].data_assets[0].name}\n"
        gdpr_mapping_string += f"Data sources: {', '.join(self.gdpr_mapping[process].data_sources)}\n"
        gdpr_mapping_string += f"Purposes: {', '.join(self.gdpr_mapping[process].purpose)}\n"
        gdpr_mapping_string += f"Recipients: {', '.join(self.gdpr_mapping[process].recipients)}\n"
        gdpr_mapping_string += f"Storage location: {', '.join(self.gdpr_mapping[process].storage_location)}\n"
        gdpr_mapping_string += f"Retention period: {', '.join(self.gdpr_mapping[process].retention_period)}\n"
        gdpr_mapping_string += "-"*45+'\n'
      return gdpr_mapping_string

    def __str__(self):
        return self.privacy_policy_string

The name and contact details of the controller and the processor acting on their behalf, as well as the processor or controller’s representative and DPO, if applicable;

A description of the various categories of users and data (including from third parties);

The categories of data recipients, including non-EU third-country recipients or international organizations;

The purpose of the processing activities;

Transfers of personal data to a third country and the identification of that third country or international organization, including documentation of suitable safeguards (where applicable);

Anticipated time limits for erasure of the various categories of data (where possible);

The technical and organizational security measures described in general terms (where possible).

In [174]:
gdpr_mapping = [
    Process(
      "Customer Onboarding",
      ["Online Registration Form"],
      ["Providing Services"],
      ["Internal Sales Team", "Third-Party CRM"],
      ["Cloud Database"],
      ["5 Years"]
  ),
    Process(
      "Customer Onboarding",
      ["Online Registration Form", "CRM System"],
      ["Providing Services", "Marketing Communications"],
      ["Internal Sales Team", "Third-Party CRM", "Marketing Department"],
      ["Cloud Database", "CRM System"],
      ["5 Years", "Until Opt-Out"]
  ),
    Process(
      "Product Development",
      ["Market Research Surveys", "Customer Feedback Forms", "Product Usage Data"],
      ["Improving Products and Services", "Developing New Features"],
      ["Product Team", "Marketing Team", "Data Analytics Team"],
      ["Product Development Database", "Data Warehouse"],
      ["Indefinite", "Until Product End-of-Life"]
  ),
    Process(
      "Order Fulfillment",
      ["E-commerce Platform", "Point-of-Sale System"],
      ["Delivering Purchased Products", "Processing Returns and Refunds"],
      ["Warehouse Team", "Shipping Providers", "Customer Service Team"],
      ["Order Management System", "Inventory Management System"],
      ["7 Years", "Until Account Closure"]
  ),
    Process(
      "Employee Recruitment",
      ["Online Job Portals", "Employee Referrals", "Recruitment Agencies"],
      ["Hiring New Employees", "Building Talent Pipeline"],
      ["HR Department", "Hiring Managers", "Recruitment Agencies"],
      ["Applicant Tracking System", "HR Database"],
      ["3 Years", "Until Candidate Withdrawal"]
  ),
    Process(
      "Invoice Processing",
      ["Vendor Information", "Payment Details", "Invoice History"],
      ["Paying Vendors for Goods and Services", "Managing Vendor Relationships"],
      ["Accounts Payable Team", "Finance Department", "Procurement Team"],
      ["Accounting Software", "Vendor Management System"],
      ["7 Years", "In Accordance with Tax Regulations"]
  )
]

In [175]:
data_assets = [
    DataAsset(
        name="Customer Database",
        source="CRM System",
        dpo="Jane Doe"
    ),
    DataAsset(
        name="Website Analytics",
        source="Google Analytics",
        dpo="John Smith"
    ),
    DataAsset(
        name="Employee Records",
        source="HR System",
        dpo="Alice Johnson"
    ),
    DataAsset(
        name="Financial Transactions",
        source="Accounting Software",
        dpo="Robert Brown"
    ),
    DataAsset(
        name="Email Marketing Lists",
        source="Mailchimp",
        dpo="Emily Davis"
    )
]

In [176]:
data_categories = [
    DataCategory(
        name="contact information"
    ),
    DataCategory(
        name="location"
    ),
    DataCategory(
        name="medical"
    ),
    DataCategory(
        name="personal information"
    ),
    DataCategory(
        name="network information"
    ),
    DataCategory(
        name="transaction data"
    ),
    DataCategory(
        name="physical attribute"
    ),
    DataCategory(
        name="authentication"
    ),
    DataCategory(
        name="financial"
    ),
    DataCategory(
        name="government id"
    ),
    DataCategory(
        name="name"
    )
]

In [177]:
data_elements = [
    DataType(
        name="Email Address",
        description="An electronic mail address used to send and receive emails.",
        category="Contact Information",
        is_personal_data=True
    ),
    DataType(
        name="IP Address",
        description="A numerical label assigned to each device connected to a computer network.",
        category="Network Information",
        is_personal_data=False
    ),
    DataType(
        name="Date of Birth",
        description="The date on which a person was born.",
        category="Personal Information",
        is_personal_data=True
    ),
    DataType(
        name="Purchase History",
        description="A record of the products and services purchased by a customer.",
        category="Transaction Data",
        is_personal_data=False
    ),
    DataType(
        name="Geolocation Data",
        description="Information that can be used to identify the physical location of a device.",
        category="Location Information",
        is_personal_data=False
    ),
    DataType(
        name="Medical Records",
        description="Documentation of a patient's medical history and care.",
        category="Health Information",
        is_personal_data=True
    )
]

In [182]:
privacy_policy.detected_data_types

[{'number': ['id'],
  'email address': ['email_address', 'personal_email'],
  'phone number': ['home_phone', 'fax_number', 'business_phone'],
  'city name': ['city'],
  'name': ['last_name', 'first_name'],
  'occupation': ['job_title'],
  'nationality': ['country_region'],
  'employment history and language skill': ['notes'],
  'unknown': ['company'],
  'address': ['address'],
  'location': ['state_province'],
  'zip code': ['zip_postal_code']},
 [],
 {'unclear': ['id'],
  'email address': ['email_address', 'personal_email'],
  'phone number': ['home_phone', 'fax_number', 'business_phone'],
  'city name': ['city'],
  'name': ['last_name', 'first_name'],
  'occupation': ['job_title'],
  'country': ['country_region'],
  'career history': ['notes'],
  'company name': ['company'],
  'address': ['address'],
  'location': ['state_province'],
  'zip code': ['zip_postal_code']},
 [],
 {'number': ['id'],
  'email address': ['email_address', 'personal_email'],
  'phone number': ['home_phone', 'f

In [181]:
privacy_policy = PrivacyPolicy()

fn = lambda pp,process: pp.add_process(process)
fn2 = lambda pp,process,asset: pp.gdpr_mapping[process].add_data_asset(asset)
fn3 = lambda pp,process,data_type: pp.gdpr_mapping[process].data_assets[0].add_data_type(data_type)
fn4 = lambda pp,process,df: pp.gdpr_mapping[process].data_assets[0].set_df_data(df)

for process in gdpr_mapping:
  fn(privacy_policy,process)

for process in privacy_policy.gdpr_mapping:
  fn2(privacy_policy,process,random.choice(data_assets))
  fn3(privacy_policy,process,random.choice(data_elements))

for process in privacy_policy.gdpr_mapping:
  for data_asset in privacy_policy.gdpr_mapping[process].data_assets:
    try:
      fn4(privacy_policy,process,df_dataset)
    except:
     continue
#privacy_policy.print_gdpr_mapping()
#privacy_policy.set_df_dataset(df_dataset)
#privacy_policy.set_detected_data_types()
#privacy_policy.set_regulation("HIPAA")

privacy_policy.set_detected_data_types()
privacy_policy.generate_privacy_policy()#lang='spanish'
print(privacy_policy.privacy_policy_string)
#privacy_policy.privacy_policy_string
#privacy_policy.gdpr_mapping[].get_data_assets()

number
email address
email address
phone number
phone number
city name
name
name
occupation
phone number
nationality
employment history and language skill
unknown
address
location
zip code
error en detect_data de asset
unclear
email address
email address
phone number
phone number
city name
name
name
occupation
phone number
country
career history
company name
address
location
zip code
error en detect_data de asset
number
email address
email address
phone number
phone number
city name
name
name
job title
phone number
country
employment history
preference
address
location datum
postal code
unknown
email address
email address
phone number
phone number
city name
name
name
occupation
phone number
country name
employment history
car brand
address
location datum
zip code
unidentifiable datum
email address
email address
phone number
phone number
city name
name
name
occupation
phone number
country name
career history and language skill
vehicle brand
address
state
postal code
unknown
email addres